# Digital-Contact-Tracing on SocioPattern

This notebook demonstrates the use of the simulator on a SocioPattern dataset.

### Imports and setup

In [1]:
import numpy as np
import DigitalContactTracing as DCT
from load_temporal_graph import load_df_socio, build_graphs_socio, get_array_of_contacts
from utils import get_graph_socio_from_csv

%load_ext autoreload
%autoreload 2

### Define the parameters of the simulation

In [2]:
import simulation_parameters_sociopattern
PARAMETERS = simulation_parameters_sociopattern.parameters_dict[1]
PARAMETERS

{'temporal_gap': 300,
 'memory_contacts': 7,
 'max_time_quar': 10,
 'R0_reduction_factor': 1.0,
 'policies': {'policy1': [1800],
  'policy2': [1200],
  'policy3': [900],
  'policy4': [600],
  'policy5': [300]},
 'eps_Is': [0.2, 0.5, 0.8, 1.0],
 'times': 10,
 'seed': 11,
 'Y_i': 1,
 'nc': 0.0,
 'symptomatics': 0.6,
 'testing': 0.5,
 'store': {'to_store': True, 'path_to_store': 'RESULTS_Socio/sim1/'}}

### Load the temporal graph

In [3]:
## High-School
graphs = get_graph_socio_from_csv('High-School', 'Dataset/High-School.csv', PARAMETERS['temporal_gap']) # The first time it's computed and saved, then it's just loaded
## InVS:
# graphs = get_graph_socio_from_csv('InVS15', 'Dataset/InVS15.dat', PARAMETERS['temporal_gap']) # The first time it's computed and saved, then it's just loaded

Graph already computed: load from memory


### Run the simulations and store the results

In [4]:
# Initialize the random seed
np.random.seed(PARAMETERS["seed"])

# Run over each configuration
for policy in PARAMETERS["policies"]:
    filter_rssi = None
    filter_duration = PARAMETERS["policies"][policy][0]
    print('Computing the policy adjacency matrix with policy: %s' % policy)
    A_policy = DCT.policy_adjacency_matrix(graphs, filter_duration, filter_rssi)
    
    for eps_I in PARAMETERS["eps_Is"]:
        results = []
        
        # Print some report
        print("-" * 100)
        print('Running simulation with policy: %s' % policy)
        print('       |_ eps_I             : %+4.2f'   % eps_I)
        print('       |_ filter_duration   : %+4.2f' % filter_duration)
        print('')
        
        # Repeat the simulation for a fixed configuration 
        for i in range(PARAMETERS["times"]):
            # Print some report
            print('       Running simulation     [%4d / %4d]' % (i + 1, PARAMETERS['times']))

            # Initialize the simulation object
            dct = DCT.DigitalContactTracing(graphs=graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration,
                                            A_policy=A_policy,
                                            use_rssi=False)
            # Run the actual simulation
            res = dct.simulate()
            
            # Save the results    
            if PARAMETERS["store"]["to_store"]:    
                DCT.store_real_time(res, PARAMETERS, filter_rssi, filter_duration, eps_I)

Computing the policy adjacency matrix with policy: policy1
----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I             : +0.20
       |_ filter_duration   : +1800.00

       Running simulation     [   1 /   10]
       Running simulation     [   2 /   10]
       Running simulation     [   3 /   10]
       Running simulation     [   4 /   10]
       Running simulation     [   5 /   10]


KeyboardInterrupt: 